In [10]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Initialize Firebase Admin
cred = credentials.Certificate(r'C:\Users\Fady Kaisar\Downloads\ML.json')


# Initialize Firebase Admin
cred = credentials.Certificate(r'C:\Users\Fady Kaisar\Downloads\ML.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://console.firebase.google.com/project/ml-soaking-app/database/ml-soaking-app-default-rtdb/data/~2F'  # Correct Firebase URL
})

# Your code continues...


# Load data from Firebase
def fetch_data_from_firebase():
    ref = db.reference('/data')  # Specify the path where your data is stored
    data = ref.get()
    if data:
        df = pd.DataFrame(data)
        print("Data loaded from Firebase")
        return df
    else:
        print("No data found in Firebase")
        return None

# Train the model (use the code you already wrote to train and save the model)
def train_model(df):
    filtered_data = df[(df['Temperature (°C)'] >= 0) & (df['Temperature (°C)'] <= 100)]
    filtered_data = filtered_data[filtered_data['Sodium_sulfide'] >= 0]

    X = filtered_data[['Temperature (°C)', 'Soaking_pH']]
    y = filtered_data['Sodium_sulfide']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Polynomial transformation
    poly = PolynomialFeatures(degree=2, interaction_only=True)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    # Train the model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_poly, y_train)

    # Model Evaluation
    y_pred = model.predict(X_test_poly)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Model Evaluation: MSE: {mse:.2f}, R²: {r2:.2f}")

    # Save the model
    joblib.dump(model, "random_forest_model.pkl")
    print("Model saved as random_forest_model.pkl")

    return model, poly

# Predict new data from Firebase using the trained model
def predict_and_upload_to_firebase(model, poly):
    # Fetch the data to make predictions
    new_data = fetch_data_from_firebase()

    if new_data is not None:
        X_new = new_data[['Temperature (°C)', 'Soaking_pH']]
        X_new_poly = poly.transform(X_new)

        # Predict using the model
        predictions = model.predict(X_new_poly)

        # Prepare the data to upload
        new_data['Predictions'] = predictions

        # Upload predictions back to Firebase
        ref = db.reference('/predictions')  # Path where predictions should be stored
        ref.set(new_data.to_dict())

        print("Predictions uploaded to Firebase.")
    else:
        print("No new data available to predict.")

# Main function to load, train, predict, and upload
def main():
    # Fetch the training data (you may use Firebase or local data for initial training)
    training_data = fetch_data_from_firebase()
    if training_data is not None:
        model, poly = train_model(training_data)
        predict_and_upload_to_firebase(model, poly)

if __name__ == "__main__":
    main()


ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.